<a href="https://colab.research.google.com/github/roscoekerby/python-projects/blob/main/Dynamic_Podcasts_Episodes_Scraper_2024_10_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Imports

In [1]:
!pip install selenium
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
G

In [3]:
!pip install webdriver-manager

# **Scrape Apple 2024-10-28**

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = '/usr/bin/google-chrome'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1920, 1080)  # Set a larger window size
    return driver

def wait_for_episodes(driver, current_count):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: len(x.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")) > current_count
        )
        return True
    except TimeoutException:
        return False

def scroll_to_bottom(driver):
    previous_episode_count = 0
    no_new_content_count = 0
    max_attempts = 10  # Increased max attempts

    while no_new_content_count < max_attempts:
        # Get current episode count
        current_episodes = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        current_count = len(current_episodes)

        if current_count == previous_episode_count:
            no_new_content_count += 1
        else:
            no_new_content_count = 0

        # Scroll in smaller increments
        if current_episodes:
            for i in range(max(0, len(current_episodes)-5), len(current_episodes)):
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", current_episodes[i])
                    time.sleep(0.5)
                except:
                    continue

        time.sleep(2)

        new_content_loaded = wait_for_episodes(driver, current_count)
        if not new_content_loaded:
            no_new_content_count += 1

        previous_episode_count = current_count
        print(f"Current episode count: {current_count}")

def extract_episode_info(episode, index):
    """Extract information from an episode element with enhanced retry logic."""
    max_retries = 5  # Increased retries
    for attempt in range(max_retries):
        try:
            # Wait for elements to be present
            WebDriverWait(episode, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.episode-details__title-text"))
            )

            title = episode.find_element(By.CSS_SELECTOR, "span.episode-details__title-text").text
            link = episode.find_element(By.CSS_SELECTOR, "a.link-action").get_attribute('href')
            description = episode.find_element(By.CSS_SELECTOR, "p.episode-details__summary").text

            if not all([title, link, description]):
                raise NoSuchElementException("Missing data")

            return {
                'index': index,
                'title': title,
                'link': link,
                'description': description
            }
        except (StaleElementReferenceException, NoSuchElementException) as e:
            if attempt == max_retries - 1:
                print(f"Failed to extract episode at index {index} after {max_retries} attempts: {str(e)}")
            time.sleep(1)
            continue
    return None

def scrape_episodes(url):
    episode_data = []
    driver = init_driver()

    try:
        driver.get(url)

        # Wait for initial load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li.svelte-8rlk6b"))
        )

        # Scroll to load all episodes
        scroll_to_bottom(driver)

        # Get all episodes after scrolling
        episode_elements = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        print(f"Total episodes found: {len(episode_elements)}")

        # Extract information from each episode
        for index, episode in enumerate(episode_elements):
            episode_info = extract_episode_info(episode, index)
            if episode_info:
                episode_data.append(episode_info)
                print(f"Scraped ({index + 1}/{len(episode_elements)}): {episode_info['title']}")
            else:
                print(f"Failed to scrape episode at index {index}")

        # Sort episodes by index to maintain order
        episode_data.sort(key=lambda x: x['index'])

        # Check for missing episodes
        titles = [ep['title'] for ep in episode_data]
        for i in range(1, 27):  # Adjust range based on expected episode count
            episode_num = f"TMGP Ep {str(i).zfill(2)}"
            if not any(episode_num in title for title in titles):
                print(f"WARNING: Potentially missing {episode_num}")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.quit()

    return episode_data

if __name__ == "__main__":
    url = "https://podcasts.apple.com/za/podcast/the-muscle-growth-podcast/id1717906577/episodes"
    apple_episodes = scrape_episodes(url)
    print(f"\nTotal episodes successfully scraped: {len(apple_episodes)}")

    # Print all episode titles in order
    print("\nAll scraped episodes:")
    for ep in apple_episodes:
        print('*' * 80)
        print(f"{ep['title']}")
        print(f"{ep['link']}")
        print(f"{ep['description']}")
        print('*' * 80)


Current episode count: 25
Current episode count: 26
Current episode count: 26
Current episode count: 26
Current episode count: 26
Current episode count: 26
Current episode count: 26
Total episodes found: 26
Scraped (1/26): TMGP Ep 26 with wellness and holistic health coach and author Chris Terzakos p2
Scraped (2/26): TMGP Ep 25 with wellness and holistic health coach and author Chris Terzakos p1
Scraped (3/26): TMGP Ep 24 with the VEGAN GIANT - Daniel Louwrens p2
Scraped (4/26): TMGP Ep 23 with 6’4’’ Vegan Bodybuilding Giant and Expert Coach/Consultant Daniel Louwrens
Scraped (5/26): TMGP Ep 22 with Injury Coach, Chiropractor and Powerlifter Dr J’aime Goguen-Locke
Scraped (6/26): TMGP Ep 21 with Muscle Genius & National Powerlifter Michael Cowley
Scraped (7/26): TMGP Ep 20 with Myology genius & National Powerlifter Michael Cowley
Scraped (8/26): TMGP Ep 19 with Male Hormone Expert Dr Moeller part 2
Scraped (9/26): TMGP Ep 18 with Male Hormone Expert Dr Moeller
Scraped (10/26): TMGP Ep 

# **Scrape Spotify**

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

# Function to initialize the WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = '/usr/bin/google-chrome'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to click elements using CSS selectors with a retry mechanism
def click_element(driver, selector, max_retries=5, wait_time=3):
    retries = 0
    while retries < max_retries:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            driver.execute_script("arguments[0].scrollIntoView();", element)  # Ensure button is visible
            element.click()
            time.sleep(wait_time)  # Add a wait time to load additional episodes
            return True
        except Exception as e:
            retries += 1
            print(f"Retry {retries}/{max_retries} - Failed to click: {e}")
            time.sleep(wait_time)
    print(f"Failed to click on element after {max_retries} retries.")
    return False

# Function to load all episodes by clicking "Load more episodes" until it's unavailable
def load_all_episodes(driver):
    load_more_selector = 'button[data-encore-id="chip"][role="checkbox"]'
    while click_element(driver, load_more_selector):
        print("Clicked 'Load more episodes' button.")
        time.sleep(3)  # Delay for loading new episodes

# Function to extract episode links from the infinite scroll list
def get_episode_links(driver):
    episode_and_show_list = []
    infinite_scroll_list = driver.find_element(By.CSS_SELECTOR, '[data-testid="infinite-scroll-list"]')
    episode_links = infinite_scroll_list.find_elements(By.TAG_NAME, 'a')

    for episode_link in episode_links:
        episode_href = episode_link.get_attribute('href')
        if episode_href and 'show' not in episode_href:
            print("Episode Link:", episode_href)
            episode_and_show_list.append(episode_href)
    return episode_and_show_list

# Main execution
if __name__ == "__main__":
    url = "https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV"
    driver = init_driver()
    driver.get(url)

    try:
        # Load all episodes by clicking "Load more episodes" until no more episodes can be loaded
        load_all_episodes(driver)

        # Extract and print episode links
        spotify_episodes = get_episode_links(driver)
        print("Total Episodes Found:", len(spotify_episodes))

    finally:
        driver.quit()


Clicked 'Load more episodes' button.
Clicked 'Load more episodes' button.
Clicked 'Load more episodes' button.
Clicked 'Load more episodes' button.
Retry 1/5 - Failed to click: Message: 
Stacktrace:
#0 0x57ee9959110a <unknown>
#1 0x57ee990ac460 <unknown>
#2 0x57ee990f8f96 <unknown>
#3 0x57ee990f9221 <unknown>
#4 0x57ee9913e5d4 <unknown>
#5 0x57ee9911d03d <unknown>
#6 0x57ee9913baf6 <unknown>
#7 0x57ee9911cdb3 <unknown>
#8 0x57ee990ebc10 <unknown>
#9 0x57ee990ecbee <unknown>
#10 0x57ee9955d39b <unknown>
#11 0x57ee99561338 <unknown>
#12 0x57ee9954aebc <unknown>
#13 0x57ee99561eb7 <unknown>
#14 0x57ee9953010f <unknown>
#15 0x57ee9957f828 <unknown>
#16 0x57ee9957f9f0 <unknown>
#17 0x57ee9958ff86 <unknown>
#18 0x7abc08a95ac3 <unknown>

Retry 2/5 - Failed to click: Message: 
Stacktrace:
#0 0x57ee9959110a <unknown>
#1 0x57ee990ac460 <unknown>
#2 0x57ee990f8f96 <unknown>
#3 0x57ee990f9221 <unknown>
#4 0x57ee9913e5d4 <unknown>
#5 0x57ee9911d03d <unknown>
#6 0x57ee9913baf6 <unknown>
#7 0x57ee991

In [6]:
spotify_episodes

['https://open.spotify.com/episode/2G9YKjM1733RO7tMJJrMMf',
 'https://open.spotify.com/episode/2ufYLgS9o9OMhF5Bb4ChQX',
 'https://open.spotify.com/episode/5sE1XbZHuRINseqmTx7Uqz',
 'https://open.spotify.com/episode/0G735vBGDMmXi6Su2GoFFu',
 'https://open.spotify.com/episode/6tPwmGJ4WKh04U82hCtGM0',
 'https://open.spotify.com/episode/6G77vKu5HMVjnxmNYWTpgm',
 'https://open.spotify.com/episode/5WY6g608AjhfF8AqQatxLb',
 'https://open.spotify.com/episode/2rUoYlIkv1SmDSWpZLhCWI',
 'https://open.spotify.com/episode/4gAbW6QEJsltCvUsfd3NdW',
 'https://open.spotify.com/episode/7ndsxdboK4dxWs5BvUFp4P',
 'https://open.spotify.com/episode/72yX4xayGrxXaIQv8NgQ3S',
 'https://open.spotify.com/episode/6oCz7AqlQDHu4ZWrn21kC3',
 'https://open.spotify.com/episode/0sWyuKGDPXiI64ycdKO7ls',
 'https://open.spotify.com/episode/58CgSrPor1Uh6mMu8z7WaI',
 'https://open.spotify.com/episode/4gzfcWJ0lfGYcDwCKf0wol',
 'https://open.spotify.com/episode/5VhoEetiuhb8C5khvN5jJz',
 'https://open.spotify.com/episode/07Nr4

In [7]:
spotify_episode_urls = []
for url in spotify_episodes:
  if "episode" in url:
    print(url)
    spotify_episode_urls.append(url)

https://open.spotify.com/episode/2G9YKjM1733RO7tMJJrMMf
https://open.spotify.com/episode/2ufYLgS9o9OMhF5Bb4ChQX
https://open.spotify.com/episode/5sE1XbZHuRINseqmTx7Uqz
https://open.spotify.com/episode/0G735vBGDMmXi6Su2GoFFu
https://open.spotify.com/episode/6tPwmGJ4WKh04U82hCtGM0
https://open.spotify.com/episode/6G77vKu5HMVjnxmNYWTpgm
https://open.spotify.com/episode/5WY6g608AjhfF8AqQatxLb
https://open.spotify.com/episode/2rUoYlIkv1SmDSWpZLhCWI
https://open.spotify.com/episode/4gAbW6QEJsltCvUsfd3NdW
https://open.spotify.com/episode/7ndsxdboK4dxWs5BvUFp4P
https://open.spotify.com/episode/72yX4xayGrxXaIQv8NgQ3S
https://open.spotify.com/episode/6oCz7AqlQDHu4ZWrn21kC3
https://open.spotify.com/episode/0sWyuKGDPXiI64ycdKO7ls
https://open.spotify.com/episode/58CgSrPor1Uh6mMu8z7WaI
https://open.spotify.com/episode/4gzfcWJ0lfGYcDwCKf0wol
https://open.spotify.com/episode/5VhoEetiuhb8C5khvN5jJz
https://open.spotify.com/episode/07Nr4mEGSerj9RMMjodO7C
https://open.spotify.com/episode/7LDBCnUkObt3rr5

# Concactenate Info from Apple and Spotify

In [8]:
def create_episode_info(apple_episodes, spotify_episode_urls):
    episodes = []

    # Ensure matching lengths between apple episodes and spotify urls
    if len(apple_episodes) != len(spotify_episode_urls):
        print("Warning: Mismatched number of Apple episodes and Spotify URLs.")
        return []

    for i in range(len(apple_episodes)):
        episode_info = {
            "title": apple_episodes[i]["title"],
            "description": apple_episodes[i]["description"],
            "urls": {
                "spotify": spotify_episode_urls[i],
                "apple": apple_episodes[i]["link"]
            }
        }
        episodes.append(episode_info)

    return episodes

episodes = create_episode_info(apple_episodes, spotify_episode_urls)


# Format into HTML for WEBSITE

In [9]:
def print_formatted_episodes_html(episodes):
    html_output = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Episodes</title>
        <style>
            body {
                font-family: Arial, sans-serif;
            }
            .episode-card {
                border: 1px solid #ddd;
                padding: 20px;
                margin: 10px 0;
                border-radius: 8px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }
            .episode-title {
                font-size: 1.5em;
                font-weight: bold;
                margin-bottom: 10px;
            }
            .episode-subtitle {
                font-size: 1.2em;
                font-weight: bold;
                color: #333;
                margin: 5px 0;
            }
            .episode-description {
                font-size: 1em;
                line-height: 1.6;
                margin: 10px 0;
            }
            .platform-link {
                font-weight: bold;
                text-decoration: none;
                margin-right: 10px;
            }
            .apple-link {
                color: #0073e6;
            }
            .spotify-link {
                color: #1DB954;
            }
        </style>
    </head>
    <body>
    <div id="episodes-container">
    """

    # Generate HTML for each episode in reverse order
    for i, episode in enumerate(reversed(episodes), 1):  # Latest to oldest order
        title = episode["title"]
        description = episode["description"]
        apple_url = episode["urls"]["apple"]
        spotify_url = episode["urls"]["spotify"]

        html_output += f"""
        <div class="episode-card">
            <div class="episode-title">Episode {i}</div>
            <div class="episode-subtitle">{title}</div>
            <div class="episode-description">{description}</div>
            <p>
                <a href="{apple_url}" target="_blank" class="platform-link apple-link">
                    🍏 Listen on Apple
                </a>
                <a href="{spotify_url}" target="_blank" class="platform-link spotify-link">
                    🎶 Listen on Spotify
                </a>
            </p>
        </div>
        """

    # Close the main container and body tags
    html_output += """
    </div>
    </body>
    </html>
    """

    print(html_output)

# Example usage:
episodes = create_episode_info(apple_episodes, spotify_episode_urls)
print_formatted_episodes_html(episodes)



    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Episodes</title>
        <style>
            body {
                font-family: Arial, sans-serif;
            }
            .episode-card {
                border: 1px solid #ddd;
                padding: 20px;
                margin: 10px 0;
                border-radius: 8px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }
            .episode-title {
                font-size: 1.5em;
                font-weight: bold;
                margin-bottom: 10px;
            }
            .episode-subtitle {
                font-size: 1.2em;
                font-weight: bold;
                color: #333;
                margin: 5px 0;
            }
            .episode-description {
                font-size: 1em;
                line-height: 1.6;
                margin: 10px 0;
            